> ## **Métodos Computacionales II - Taller #5**

**Hecho por:** Thomas Andrade Hernández (202214695) & Juan Carlos Rojas V. (202214037)

---

In [1]:
# Zona de librerías:
%matplotlib notebook
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from tqdm import tqdm
import random as rnd
import math as math
import scipy as sci

---

**1. Ecuación Diferencial de Poisson:** Resolver la ecuación diferencial de Poisson:

$$\nabla^{2}u(x, y) = e^{-xy}$$

en un dominio cuadrado $(x, y) \in [0, 20] \times [0, 20]$ cm$^{2}$, con un paso espacial de $h = 0.1$ cm. El problema se encuentra sujeto a las condiciones de frontera:

$$h_{1}(0, y) = 10 \text{ V}$$

$$h_{2}(20, y) = 0 \text{ V}$$

$$h_{3}(x, 0) = 0 \text{ V}$$

$$h_{4}(x, 20) = 0 \text{ V}$$

---

---

**2. Conducción de calor 2D:** Resolver la ecuación diferencial de conducción de calor $2D$ en el siguiente dominio rectangular $(x, y) \in [0, 1] \times [0, 1]$, $t \geq 0$. $\alpha = \beta = 1$, la discretización es $\Delta{x} = \Delta{y} = 0.2$ y $\Delta{t} = 0.1$. $T_{max} = 1$ s. La ecuación diferencial tiene las siguientes condiciones:

(a) Condición inicial:

$$u(0, x, y) = \sin(\pi(x + y))$$

(b) Condiciones de frontera:

$$u(0, y, t) = e^{-2\pi^{2}t}\sin(\pi{y})$$

$$u(x, 0, t) = e^{-2\pi^{2}t}\sin(\pi{x})$$

$$u(x, 1, t) = e^{-2\pi^{2}t}\sin(\pi{(1 + x)})$$

$$u(1, y, t) = e^{-2\pi^{2}t}\sin(\pi{(1 + y)})$$

---

---

**3. Ecuación de Onda 2D, Coordenadas Cilíndricas:** Muestre que la ecuación que describe el método de solución de la ecuación de onda en dos dimensiones considerando coordenadas cilíndricas tiene la forma:

$$u_{i, j}^{l + 1} = \nu^{2} \cdot \left[u_{i + 1, j}^{l} - 2u_{i, j}^{l} + u_{i - 1, j}^{l} + \frac{\Delta\rho}{\rho[i]} \cdot \left(u_{i, j}^{l} - u_{i - 1, j}^{l}\right) + \left(\frac{\lambda}{\rho[i]}\right)^{2} \cdot \left(u_{i, j + 1}^{l} - 2u_{i, j}^{l} + u_{i, j - 1}^{l}\right)\right] + 2u_{i, j}^{l} - u_{i, j}^{l - 1}$$

Dados

$$\lambda := \frac{\Delta\rho}{\Delta\phi} \quad \text{ y } \quad \nu := \frac{\alpha\Delta{t}}{\Delta\rho}$$

---

---

**5. Ecuación de Onda en 2D:** Resuelva la ecuación de onda en dos dimensiones:

$$\frac{\partial^{2}u(x, y, t)}{\partial{t}^{2}} = v^{2} \cdot \left(\frac{\partial^{2}u(x, y, t)}{\partial{x}^{2}} + \frac{\partial^{2}u(x, y, t)}{\partial{y}^{2}}\right)$$

donde $v = 2$ m$/$s, con $(x, y) \in R := [0, 2] \times [0, 2]$ m$^{2}$ y $t \geq 0$; con la condición inicial:

$$u(x, y, 0) = \sin(\pi{x})\sin(\pi{y})$$

La velocidad de la onda en $t = 0$ es cer0. Notar que la condición de contorno establece que las fronteras no cambian en el tiempo.

---